This notebook uses feature detection results from streetview images, elevation and classification rasters of building facades derived from lidar data, to calculate FFH.

## Load modules

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import os
import glob
from PIL import Image
# from GSV.geometry import extract_feature_pixels_lowest_region, calculate_height_difference, calculate_width_difference, estimate_FFH, estimate_FFE
import geopandas as gpd
import matplotlib.pyplot as plt
from sklearn.metrics import root_mean_squared_error, mean_absolute_error
from tqdm import tqdm
import pandas as pd
import numpy as np
from scipy.ndimage import label, generate_binary_structure

## Set input parameters
* building points containing ground elevation data
* object detection results from streetview panoramas, containing buildnig ID and panorama ID
* folder containing building facade elevation and classifcation rasters

In [ ]:
building_points_file=r'/home/ubuntu/lavender_floor_height/output/Final_Wagga_training_samples_pano_metadata_clipping_elevations.geojson'
predictions_file=r''
las_projections_folder=r"/mnt/floorheightvolume/lidar_Wagga/clipped_projected/"

## Set outputs

## Read feature detection results

## Read in building points

In [ ]:
gdf_building_points=gpd.read_file(building_points_file)
gdf_building_points=gdf_building_points[gdf_building_points["USAGE"]=="Residential"].reset_index(drop=True)
gdf_building_points

## Test calculation with one example
#### Extract ground elevations across building and surrounding, derived from lidar

In [ ]:
i=0
gdf_building_points.iloc[i]['lidar_elev_min']

In [ ]:
ground_elevation_overall = gdf_building_points.iloc[i]['lidar_elev_25pct']
ground_elevation_overall

#### Find corresponding projected rasters

In [ ]:
building_ufi=gdf_building_points.iloc[i]['UFI']
elevation_file_path=glob.glob(las_projections_folder+'*'+'_UFI_'+str(building_ufi)+'_elevation_resampled.tif')[0]
classification_file_path=glob.glob(las_projections_folder+'*'+'_UFI_'+str(building_ufi)+'_classification_resampled.tif')[0]
classification_file_path

### Calculate feature properties
* metrics for filtering: width, height, area and width/heigh ratio

### Filter detected features
* Keep the front door with closest to standard feature metrics
* Keep the lowest and most confident feature for the other classes

#### Calculate FFH based on rules

In [ ]:
def get_closest_ground_area_optimized(class_raster, elev_raster, x, y, min_area=5):
    """
    Optimized version that stops searching once the closest valid ground area is found.
    """
    if class_raster[y, x] == 2:
        return None

    # Label ground areas only below the target pixel
    struct = generate_binary_structure(2, 2)
    mask = np.zeros_like(class_raster, dtype=bool)
    mask[y+1:, :] = True  # Only search below
    labeled_ground, _ = label((class_raster == 2) & mask, structure=struct)

    closest_info = None
    min_distance = float('inf')

    # Scan row by row downward (closest first)
    for gy in range(y + 1, class_raster.shape[0]):
        row_checked = False
        current_ydist = gy - y
        
        # Early exit if we can't possibly find a closer area
        if current_ydist > min_distance:
            break

        for gx in range(class_raster.shape[1]):
            label_id = labeled_ground[gy, gx]
            if label_id > 0 and not row_checked:
                # Get full area stats if this row might contain a closer area
                ground_mask = (labeled_ground == label_id)
                area_size = np.sum(ground_mask)
                
                # Only process if meets area threshold
                if area_size >= min_area:
                    # Find closest point in this area
                    yy, xx = np.where(ground_mask)
                    distances = np.sqrt((yy - y)**2 + (xx - x)**2)
                    idx = np.argmin(distances)
                    current_dist = distances[idx]
                    
                    if current_dist < min_distance:
                        elev_values = elev_raster[ground_mask]
                        closest_info = {
                            'elev_diff': elev_raster[y, x] - np.mean(elev_values),
                            'median_diff': elev_raster[y, x] - np.median(elev_values),
                            'std_dev': np.std(elev_values),
                            'area_size': area_size,
                            'distance': current_dist,
                            'closest_y': yy[idx],
                            'closest_x': xx[idx]
                        }
                        min_distance = current_dist
                row_checked = True  # Skip other pixels in this labeled area

    return closest_info

In [ ]:
classification_arr=np.array(Image.open(classification_file_path))
elevation_arr=np.array(Image.open(elevation_file_path))
x=1401
y=1470
min_area=5
get_closest_ground_area_optimized(class_raster=classification_arr, elev_raster=elevation_arr,x=x,y=y,min_area=min_area)

In [ ]:
elevation_arr[y,x]

In [ ]:
elevation_arr[y,x] - ground_elevation_overall

## Batch calculation for all buildings